Обучите любую модель классификации на датасете IRIS до применения PCA и после него. Сравните качество классификации по отложенной выборке.
Напишите свою реализацию метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().

In [14]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [23]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
Y = iris.target 
X.shape

(150, 4)

In [24]:
# Для начала масштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование — вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [25]:
# Найдем собственные векторы и собственные значения (англ. Eigenvalues)
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(437.77467247979905, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.1045707202107, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.01353133569719, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292897, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [26]:

eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329986, 22.85076178670179, 3.668921889282866, 0.517870910715483]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [27]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперь оценим кумулятивную, то есть накапливаемую, дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329986, 22.85076178670179, 3.668921889282866, 0.517870910715483]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [28]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [29]:
# Сформируем новую матрицу «объекты-признаки»
Z = X_.dot(W)

In [30]:
X_train_before, X_test_before, y_train_before, y_test_before = train_test_split(X_, Y, test_size=0.3)
X_train_after, X_test_after, y_train_after, y_test_after = train_test_split(Z, Y, test_size=0.3)

In [32]:
before_pca = RandomForestClassifier(max_depth=10, n_estimators=20)
before_pca.fit(X_train_before, y_train_before)
y_pred_before = before_pca.predict(X_test_before)
print(accuracy_score(y_test_before, y_pred_before))

0.9333333333333333


In [34]:
after_pca = RandomForestClassifier(max_depth=10, n_estimators=20)
after_pca.fit(X_train_after, y_train_after)
y_pred_after = after_pca.predict(X_test_after)
print(accuracy_score(y_test_after, y_pred_after))

0.9111111111111111


Напишите свою реализацию метода главных компонент посредством сингулярного разложения с использованием функции numpy.linalg.svd().

In [ ]:
def get_pca_using_SVD(X, n)
    # Compute covariance matrix
    C = np.dot(X.T, X) / (n-1)
    # Eigen decomposition
    eigen_vals, eigen_vecs = np.linalg.eig(C)
    # SVD
    U, Sigma, Vh = np.linalg.svd(X, 
        full_matrices=False, 
        compute_uv=True)
    # Relationship between singular values and eigen values:
    return np.allclose(np.square(Sigma) / (n - 1), eigen_vals)